# Dilepton analysis  

This is an example of a simple dilepton analysis, quite similar to the notebook called "Dilepton_analysis_noData.ipynb", but with some differences. The most obvious difference is that we here also include real data. This example also has a slightly more advanced event selection.  

**Notice:** This is *only an example* on how to do this. Feel free to be creative, and to find better and/or more elegant ways of doing the various steps! 

## 1. Initialization

In [1]:
%jsroot on

In [2]:
// Creates a TChain to be used by the Analysis.C class
#include <TChain.h>
#include <vector>
#include <TFile.h>
#include <iostream>
#include <string>
#include <stdio.h>

In [3]:
// Sets the verbosity
DEBUG = 0;

**Importarnt**: We need to download some of the data and MC from the [ATLAS OpenData pages](http://opendata.atlas.cern/release/2020/documentation/datasets/files.html). The example below uses the data sets requiring exactly two leptons. Specify the path where you saved the datasets in the cell below:

In [4]:
// Set the path to where data and MC is stored
const char *inputpath_MC = "/ATLAS_opendata/2lep/MC/";
const char *inputpath_DATA = "/ATLAS_opendata/2lep/Data/";

In [5]:
TChain *background = new TChain("mini");
TChain *data = new TChain("mini"); 

In [6]:
// Reads the information (cross-section, number of simulated events etc.) needed to get the scaling right
std::map<TString,std::pair<double,double> > SF;
string name, xsec, sumw, eff;
std::string ifile;
ifile = "../Input/Files_base.txt";
ifstream input(ifile.c_str());
std::string line;
while(getline(input,line)){
    if (line.find("#") != line.npos )continue; // a # is a comment and not read
    istringstream linestream(line);
    std::getline(linestream, name, '|');
    std::getline(linestream, xsec, '|');
    std::getline(linestream, sumw, '|');
    std::getline(linestream, eff);
    if(DEBUG)cout << name << " " << xsec << " " << sumw << " " << eff << endl;
    float sumw_eff = atof(sumw.c_str()) * atof(eff.c_str());
    SF[TString(name)] = std::make_pair(atof(xsec.c_str()), sumw_eff);    
}

In [7]:
// We also need to categorise the various background files into categories (ttbar,diboson,singletop etc.)
vector<TString> Backgrounds; 
std::map<TString,std::pair<TString,int> > samples;
string name, cat, dsid;
std::string ifile;
ifile = "../Input/Infofile.txt";
ifstream input(ifile.c_str());
std::string line;
while(getline(input,line)){
    if (line.find("#") != line.npos )continue; // a # is a comment and not read
    istringstream linestream(line);
    std::getline(linestream, name,'|');
    std::getline(linestream, cat,'|');
    std::getline(linestream, dsid,'|');
    if(DEBUG)cout << name << " " << cat << " " << dsid << endl;
    samples[TString(name)] = std::make_pair(TString(cat), atof(dsid.c_str()));
    if(!(std::count(Backgrounds.begin(), Backgrounds.end(), TString(cat)))){
    Backgrounds.push_back(TString(cat));
    }
    //float sumw_eff = atof(sumw.c_str()) * atof(eff.c_str());
    //SF[TString(name)] = std::make_pair(atof(xsec.c_str()), sumw_eff);    
}

In [8]:
if(DEBUG){
    for (const auto& kv: samples) {
        std::cout << "Key: " << kv.first << " Value: " << (kv.second).first << ", " << (kv.second).second<<std::endl;
    }
}

In [9]:
// Finds all the background files in the path specified above and categorised
const char *ext=".root";
int nfiles_MC = 0;
TSystemDirectory dir(inputpath_MC, inputpath_MC); 
TList *files = dir.GetListOfFiles(); 
if (files) { 
    TSystemFile *file; 
    TString fname; 
    TIter next(files); 
    while ((file=(TSystemFile*)next())) { 
        fname = file->GetName(); 
        if (!file->IsDirectory() && fname.EndsWith(ext)) 
        {
            //cout << fname.Data() << endl; 
            TObjArray *tx = fname.Tokenize(".");
            if(samples.find(((TObjString *)(tx->At(1)))->String()) == samples.end()){
                if(DEBUG)cout<<"File "<<fname<<" not included in analysis"<<endl;
                continue;
            }
            if(DEBUG)cout<<"Adding "<<TString(inputpath_MC)+fname<<endl;
            background->Add(TString(inputpath_MC)+fname);
            nfiles_MC += 1;
            
        } 
    } 
} 
printf("MC: Added %i files with %lld events\n",nfiles_MC,background->GetEntries());

MC: Added 115 files with 66891691 events


In [10]:
// Reads all available data
const char *ext=".root";
TSystemDirectory dir(inputpath_DATA, inputpath_DATA); 
TList *files = dir.GetListOfFiles(); 
int nfiles_data = 0;
if (files) { 
    TSystemFile *file; 
    TString fname; 
    TIter next(files); 
    while ((file=(TSystemFile*)next())) { 
        fname = file->GetName(); 
        if (!file->IsDirectory() && fname.EndsWith(ext)) 
        {
            //cout << fname.Data() << endl; 
            TObjArray *tx = fname.Tokenize(".");
            if(DEBUG)cout<<"Adding "<<TString(inputpath_MC)+fname<<endl;
            data->Add(TString(inputpath_DATA)+fname);
            nfiles_data += 1;
            
        } 
    } 
} 
printf("DATA: Added %i files with %lld events\n",nfiles_data,data->GetEntries());

DATA: Added 4 files with 12205790 events


## 2. Reading the dataset

Next we define the variables we want to include in the analysis, and link them to branches in the TTree. A few things to notice at this point: 
-  In this example we will only study events with two leptons, so the vectorial variables only need to be two dimensional. 
-  The variables are here given names corresponding to the branches in the TTree. This is not necessary, so if you want to give them other names you are free to do so. 
-  The variable called "channelNumber" is the same as we have called "dataset ID" above. These terms are used interchangeably. 

In [11]:
// It is important to set the pointers to zero. If not you will get a segmentation violation when running
 unsigned int lep_n;
  vector<int> *lep_charge = 0;
  int channelNumber;
  vector<UInt_t> *lep_type = 0;
  vector<float> *lep_pt = 0;
  vector<float> *lep_E = 0;
  vector<float> *lep_phi = 0;
  vector<float> *lep_eta = 0;
  vector<float> *jet_MV2c10 = 0;
  vector<float> *lep_etcone20 = 0;
  vector<float> *lep_ptcone30 = 0;  
  float met_et;
  vector<bool> *lep_trigMatched = 0;
  bool trigE;
  bool trigM; 

In [12]:
float scaleFactor_PILEUP; 
float scaleFactor_ELE;
float scaleFactor_MUON;
float scaleFactor_BTAG;
float scaleFactor_LepTRIGGER;
float mcWeight;

In [13]:
 background->SetBranchAddress("lep_n",      &lep_n);
 
  background->SetBranchAddress("lep_charge", &lep_charge);
  background->SetBranchAddress("lep_type",   &lep_type);
  background->SetBranchAddress("lep_pt",     &lep_pt);
  background->SetBranchAddress("lep_eta",    &lep_eta);
  background->SetBranchAddress("lep_phi",    &lep_phi);
  background->SetBranchAddress("lep_E",      &lep_E);
  background->SetBranchAddress("met_et",     &met_et); 
  background->SetBranchAddress("channelNumber", &channelNumber);
  background->SetBranchAddress("mcWeight", &mcWeight); 
  background->SetBranchAddress("scaleFactor_PILEUP", &scaleFactor_PILEUP ); 
  background->SetBranchAddress("scaleFactor_ELE", &scaleFactor_ELE ); 
  background->SetBranchAddress("scaleFactor_MUON", &scaleFactor_MUON ); 
  background->SetBranchAddress("scaleFactor_BTAG", &scaleFactor_BTAG ); 
  background->SetBranchAddress("scaleFactor_LepTRIGGER", &scaleFactor_LepTRIGGER ); 
  background->SetBranchAddress("lep_ptcone30", &lep_ptcone30); 
  background->SetBranchAddress("lep_etcone20", &lep_etcone20); 
  background->SetBranchAddress("trigE", &trigE); 
  background->SetBranchAddress("trigM", &trigM); 
  background->SetBranchAddress("jet_MV2c10", &jet_MV2c10); 

  // For data 
  data->SetBranchAddress("lep_n",      &lep_n);
  data->SetBranchAddress("lep_charge", &lep_charge);
  data->SetBranchAddress("lep_type",   &lep_type);
  data->SetBranchAddress("lep_pt",     &lep_pt);
  data->SetBranchAddress("lep_eta",    &lep_eta);
  data->SetBranchAddress("lep_phi",    &lep_phi);
  data->SetBranchAddress("lep_E",      &lep_E);
  data->SetBranchAddress("met_et",     &met_et); 
  data->SetBranchAddress("channelNumber", &channelNumber);
  data->SetBranchAddress("trigE", &trigE); 
  data->SetBranchAddress("trigM", &trigM); 
  data->SetBranchAddress("lep_trigMatched", &lep_trigMatched); 
  data->SetBranchAddress("lep_ptcone30", &lep_ptcone30); 
  data->SetBranchAddress("lep_etcone20", &lep_etcone20); 
  data->SetBranchAddress("jet_MV2c10", &jet_MV2c10); 

## 3. Making (a lot of) histograms

Now that we have read our dataset we want to start analyzing the data. To do so we need to put the data into histograms. For reasons that will become clear later in the analysis we must (for each variable) make one histogram per dataset ID. (We have 31 background samples, so if we want to study 10 variables we have to make 310 histograms!) A very elegant way of dealing with all these histograms is by using [map](http://www.cplusplus.com/reference/map/map/)s (the C$++$ equivalent of Python dictionaries). Below we define one map for each variable. Here the *key values* are the dataset IDs, while the *mapped values* are the histograms.   

In [14]:
map<Int_t, TH1*> hist_mll; 
map<Int_t, TH1*> hist_lep_pt; 
map<Int_t, TH1*> hist_met;

In [15]:
for (const auto& kv: samples) {
    int dsid = (kv.second).second;
    hist_mll[dsid] = new TH1F(); 
    hist_lep_pt[dsid] = new TH1F(); 
    hist_met[dsid] = new TH1F();
    hist_mll[dsid]->SetNameTitle("hist_mll", "Invariant mass"); 
    hist_lep_pt[dsid]->SetNameTitle("hist_lep_pt", "Lepton pT"); 
    hist_met[dsid]->SetNameTitle("hist_met", "Missing ET");
    hist_mll[dsid]->SetBins(20,0,500); 
    hist_lep_pt[dsid]->SetBins(20,0,1000);
    hist_met[dsid]->SetBins(20,0,500); 
}

In [16]:
TH1F *hist_mll_d = new TH1F(); 
TH1F *hist_lep_pt_d = new TH1F(); 
TH1F *hist_met_d = new TH1F(); 
hist_mll_d->SetNameTitle("hist_mll", "Invariant mass"); 
hist_lep_pt_d->SetNameTitle("hist_lep_pt", "Lepton pT"); 
hist_met_d->SetNameTitle("hist_met", "Missing ET");
hist_mll_d->SetBins(20,0,500); 
hist_lep_pt_d->SetBins(20,0,1000);
hist_met_d->SetBins(20,0,500); 

### 2.1 Fill the histograms 
We can now loop over all events in our dataset, implement desired cuts, and fill the histograms we created above. In this example we choose only events containing exactly to same flavour leptons with opposite charge (i.e. $e^+e^-$ or $\mu^+\mu^-$). 
Before starting the loop we extract the total number of entries (events) in the TChain. We also make [TLorentzVector](https://root.cern.ch/doc/master/classTLorentzVector.html)s, which are very practical for handling the kinematics of the leptons, e.g. calculating the invariant mass of the two leptons. 

In [17]:
TLorentzVector l1, l2, dileptons; 

In [18]:
TChain *dataset = new TChain("mini"); 
int isData = 0; 
int nentries; 
Float_t W; 
int n_bjets; 

In [19]:
// Reset histograms (in case you have filled them before) 
for(const auto& kv: samples){ 
    int dsid = (kv.second).second;
    hist_mll[dsid]->Reset(); 
    hist_lep_pt[dsid]->Reset(); 
    hist_met[dsid]->Reset();
}

hist_mll_d->Reset(); 
hist_lep_pt_d->Reset(); 
hist_met_d->Reset(); 
std::cout<<"isData = "<<isData<<std::endl;

isData = 0


Below is the main cell for the analysis. 

In [20]:
isData = 0;
for (Long64_t i = 0; i < background->GetEntries(); i++){
    
    if( i%1000000 == 0 && i>0){ std::cout << i/1000000 << " million events processed" << std::endl;}
    
    background->GetEntry(i);
     // Require "good leptons":
    /**
      if(lep_n != 2){ continue; }

      if( lep_pt->at(0)/1000.0 < 25 ){ continue; }

    
      if( lep_etcone20->at(0)/lep_pt->at(0) > 0.15 ){ continue; }
      if( lep_ptcone30->at(0)/lep_pt->at(0) > 0.15 ){ continue; }

      if( lep_pt->at(1)/1000.0 < 25 ){ continue; }
      if( lep_etcone20->at(1)/lep_pt->at(1) > 0.15 ){ continue; }
      if( lep_ptcone30->at(1)/lep_pt->at(1) > 0.15 ){ continue; }
*/
      // Event selection: 
      // Cut #1: Require (exactly) 2 leptons
    cout<<"lepN = "<<lep_n<<endl;
      if(lep_n != 2){ continue; }
    cout<<"charge = "<<lep_charge->at(0)<<endl;
      // Cut #2: Require opposite charge
      if(lep_charge->at(0) == lep_charge->at(1)){ continue; }
      // Cut #3: Require same flavour (2 electrons or 2 muons)
      if(lep_type->at(0) != lep_type->at(1)){ continue; }

      // Set Lorentz vectors: 
      l1.SetPtEtaPhiE(lep_pt->at(0)/1000., lep_eta->at(0), lep_phi->at(0), lep_E->at(0)/1000.);
      l2.SetPtEtaPhiE(lep_pt->at(1)/1000., lep_eta->at(1), lep_phi->at(1), lep_E->at(1)/1000.);
      // Variables are stored in the TTree with unit MeV, so we need to divide by 1000 
      // to get GeV, which is a more practical and commonly used unit. 

      dileptons = l1 + l2;
    Float_t scaleFactor = 1.0;
    if(!isData){
        scaleFactor = mcWeight*scaleFactor_ELE*scaleFactor_MUON*scaleFactor_LepTRIGGER*scaleFactor_PILEUP;
    }
    if(isData){
        hist_mll_d->Fill(dileptons.M(),scaleFactor);
        hist_lep_pt_d->Fill(l1.Pt(),scaleFactor);
        hist_lep_pt_d->Fill(l2.Pt(),scaleFactor);
        hist_met_d->Fill(met_et/1000.,scaleFactor);
    }else{
        
        hist_mll[channelNumber]->Fill(dileptons.M(),scaleFactor);
        hist_lep_pt[channelNumber]->Fill(l1.Pt(),scaleFactor);
        hist_lep_pt[channelNumber]->Fill(l2.Pt(),scaleFactor);
        hist_met[channelNumber]->Fill(met_et/1000.,scaleFactor);
    }
    if(i>10000)break;
}

lepN = 2
charge = 1
lepN = 2
charge = 1
lepN = 2
charge = -1
lepN = 2
charge = 1
lepN = 2
charge = 1
lepN = 2
charge = 1
lepN = 2
charge = -1
lepN = 2
charge = -1
lepN = 2
charge = 1
lepN = 2
charge = -1
lepN = 2
charge = 1
lepN = 2
charge = 1
lepN = 2
charge = -1
lepN = 2
charge = -1
lepN = 2
charge = 1
lepN = 2
charge = -1
lepN = 2
charge = -1
lepN = 2
charge = 1
lepN = 2
charge = -1
lepN = 2
charge = 1
lepN = 2
charge = 1
lepN = 2
charge = 1
lepN = 2
charge = 1
lepN = 2
charge = 1
lepN = 2
charge = 1
lepN = 2
charge = 1
lepN = 2
charge = -1
lepN = 2
charge = -1
lepN = 2
charge = 1
lepN = 2
charge = -1
lepN = 2
charge = -1
lepN = 2
charge = -1
lepN = 2
charge = -1
lepN = 2
charge = -1
lepN = 2
charge = -1
lepN = 2
charge = 1
lepN = 2
charge = -1
lepN = 2
charge = 1
lepN = 2
charge = -1
lepN = 2
charge = -1
lepN = 2
charge = -1
lepN = 2
charge = 1
lepN = 2
charge = 1
lepN = 2
charge = 1
lepN = 2
charge = -1
lepN = 2
charge = 1
lepN = 2
charge = -1
lepN = 2
charge = 1
lepN = 2
charge =

We have now done the "heavy lifting" of an analysis, i.e. looping through all the events. Usually in such an analysis we create new ROOT files where we store the histograms we made above, and then analyse the output in a separate program/script. The advantage of doing this is that you can do the rest of the analysis in another language, e.g. Python, since we are done with part that requires the speed of C$++$. If you want to write ROOT files you can check out the [TFile](https://root.cern.ch/doc/master/classTFile.html) class reference. In this example we will however carry on in C$++$. But even if you only work in C++ it can be practical to write the histograms to file, so that you don't need to run the loop _every time_ you open the notebook.  

## 4. Scale and classify the histograms (MC only) 

Before we are ready to make plots we need to do some further processing of the histograms we made above. The information necessary for doing the two steps below is found in the file **Infofile.txt**.   
1. We need to **scale** the histograms to the right cross section and luminosity. Why? When making the MC samples a certain number of events is simulated, which will usually not correspond to the number of events in our data. The expected number of events from a certain kind of process is given by $N=\sigma L$, where $\sigma$ is the cross section and $L$ is the integrated luminosity. Therefore we need to scale each histogram by a scale factor <br> <br>
$$sf = \frac{N}{N_{MC}} = \frac{ \sigma L }{N_{MC}},$$ <br>  where $N_{MC}$ is the number of generated MC events.  <br> <br>
2. We also need to **classify** the background processes into different categories. This is necessary when we eventually want to make the characteristic colorful background plots you might have seen before.  

### 4.1 Make new histograms 
Maybe a bit depressingly we have to make a set of new histograms, this time corresponding to the different background categories, instead of the dataset IDs. Notice that these new histograms are made in a very similar way as above, i.e. with the same range and binning. 

In [21]:
map<TString, TH1*> H_mll; 
map<TString, TH1*> H_lep_pt; 
map<TString, TH1*> H_met;

In [22]:
for(const auto & kv:samples){
    cat = (kv.second).first;
    H_mll[cat] = new TH1F(); 
    H_lep_pt[cat] = new TH1F(); 
    H_met[cat] = new TH1F(); 
    H_mll[cat]->SetNameTitle("hist_mll", "Invariant mass"); 
    H_lep_pt[cat]->SetNameTitle("hist_lep_pt", "Lepton pT"); 
    H_met[cat]->SetNameTitle("hist_met", "Missing ET");
    H_mll[cat]->SetBins(20,0,500); 
    H_lep_pt[cat]->SetBins(20,0,1000);
    H_met[cat]->SetBins(20,0,500); 
}

In [23]:
float L = 1000.6; 

In [24]:
for(const auto & kv:samples){
    cat = (kv.second).first;
    H_mll[cat]->Reset(); 
    H_lep_pt[cat]->Reset(); 
    H_met[cat]->Reset();
}

In [25]:
double sf;
std::pair<TString,int> pairs;
for(const auto & kv:SF){
    sf = (kv.second).first*L/(kv.second).second;  
    TString name = kv.first;
      
    // Check that it is in the samples we want to include
    if ( samples.find(name) == samples.end() ) {
     cout<<"Could not find "<<name<<" in samples dic"<<endl;  
     continue;
    }
    // Get category and dsid
    pairs = samples.at(name);
    TString type = pairs.first;
    int dsid = pairs.second;
    
    // Scale dsid histogram
    hist_mll[dsid]->Scale(sf); 
    hist_lep_pt[dsid]->Scale(sf); 
    hist_met[dsid]->Scale(sf); 
    // Add dsid histogram to correct category historgram
    H_mll[type]->Add(hist_mll[dsid]); 
    H_lep_pt[type]->Add(hist_lep_pt[dsid]); 
    H_met[type]->Add(hist_met[dsid]); 
    
}

Could not find Wminusenu in samples dic
Could not find Wminusmunu in samples dic
Could not find Wminustaunu in samples dic
Could not find Wplusenu in samples dic
Could not find Wplusmunu in samples dic
Could not find Wplustaunu in samples dic
Could not find Zee in samples dic
Could not find Zmumu in samples dic
Could not find Ztautau in samples dic
Could not find data in samples dic
Could not find ttbar_lep in samples dic


### 4.3 Color the histograms 
Make yet another map, this time containing the colors you want the backgrounds to have, and then set the colors of your histograms. Note that colors are defined by integers in ROOT. If you are not happy with the colors chosen below you can have look at the [TColor](https://root.cern.ch/doc/master/classTColor.html) class reference for more options. 

In [26]:
map<TString, Int_t> colors; 

In [28]:
colors["Diboson"] = kGreen; 
colors["Zjets"] = kYellow; 
colors["ttbar"] = kRed;
colors["singleTop"] = kBlue-7; 
colors["Wjets"] = kBlue+3; 
colors["topX"] = kOrange+1; 
colors["Higgs"] = kMagenta; 

In [30]:
for(const auto h:Backgrounds){
    H_mll[h]->SetFillColor(colors[h]); 
    H_met[h]->SetFillColor(colors[h]);
    H_lep_pt[h]->SetFillColor(colors[h]);
    
    H_mll[h]->SetLineColor(colors[h]); 
    H_met[h]->SetLineColor(colors[h]);
    H_lep_pt[h]->SetLineColor(colors[h]);
}

## 4. Stack and plot the histograms

Finally we have arrived to the part where we can plot the results of all the work done above. For each variable we need to stack the backgrounds on top of each other, which is done by using the [THStack](https://root.cern.ch/doc/master/classTHStack.html) class. In the example below we do this for two variables; invariant mass and missing $E_T$.  

In [31]:
THStack *stack_mll = new THStack("Invariant mass", "");
THStack *stack_met = new THStack("Missing ET", ""); 
THStack *stack_lep_pt = new THStack("Lepton pT", ""); 

In [33]:
for(const auto h: Backgrounds){
    stack_mll->RecursiveRemove(H_mll[h]); // Remove previously stacked histograms  
    stack_met->RecursiveRemove(H_met[h]);
    stack_lep_pt->RecursiveRemove(H_lep_pt[h]);
    stack_mll->Add(H_mll[h]); 
    stack_met->Add(H_met[h]);
    stack_lep_pt->Add(H_lep_pt[h]); 
}    

Now we make a legend (see [TLegend](https://root.cern.ch/doc/master/classTLegend.html)), and add  the different backgrounds. Next we make a canvas (see [TCanvas](https://root.cern.ch/doc/master/classTCanvas.html)), which is allways necessary when we want to make a plot. Then you draw the stack and the legend, and display them by drawing the canvas. We can also specify axis labels and a bunch of other stuff. 

In [34]:
gStyle->SetLegendBorderSize(0); // Remove (default) border around legend 
TLegend *leg = new TLegend(0.65, 0.60, 0.9, 0.85); 

In [40]:
leg->Clear();
for(const auto h: Backgrounds){
    leg->AddEntry(H_mll[h], h, "f");  // Add your histograms to the legend
} 
leg->AddEntry(hist_mll_d, "Data", "lep"); 

In [41]:
TCanvas *C = new TCanvas("c", "c", 600, 600);

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c


In [42]:
gPad->SetLogy(); // Set logarithmic y-axis

In [43]:
hist_mll_d->SetLineColor(kBlack); 
hist_mll_d->SetMarkerStyle(kFullCircle); 
hist_mll_d->SetMarkerColor(kBlack); 

In [44]:
stack_mll->Draw("hist"); 
stack_mll->SetMaximum(1E6); 
stack_mll->GetYaxis()->SetTitle("# events");
stack_mll->GetYaxis()->SetTitleOffset(1.3); 
stack_mll->GetXaxis()->SetTitle("m_{ll} (GeV)");
stack_mll->GetXaxis()->SetTitleOffset(1.3);
hist_mll_d->Draw("same E"); 
leg->Draw();
C->Draw();

In [ ]:
hist_met_d->SetLineColor(kBlack); 
hist_met_d->SetMarkerStyle(kFullCircle); 
hist_met_d->SetMarkerColor(kBlack); 

In [ ]:
stack_met->Draw("hist"); 
stack_met->SetMaximum(1E6); 
stack_met->SetMinimum(1E-2); 
stack_met->GetYaxis()->SetTitle("# events");
stack_met->GetYaxis()->SetTitleOffset(1.3); 
stack_met->GetXaxis()->SetTitle("E_{T}^{miss} (GeV)");
stack_met->GetXaxis()->SetTitleOffset(1.3);
hist_met_d->Draw("same e"); 
leg->Draw();
C->Draw(); 

In [ ]:
hist_lep_pt_d->SetLineColor(kBlack); 
hist_lep_pt_d->SetMarkerStyle(kFullCircle); 
hist_lep_pt_d->SetMarkerColor(kBlack); 

In [ ]:
stack_lep_pt->Draw("hist"); 
stack_lep_pt->SetMaximum(1E6); 
stack_lep_pt->GetYaxis()->SetTitle("# events");
stack_lep_pt->GetYaxis()->SetTitleOffset(1.3); 
stack_lep_pt->GetXaxis()->SetTitle("p_{T} (GeV)");
stack_lep_pt->GetXaxis()->SetTitleOffset(1.3);
hist_lep_pt_d->Draw("same e"); 
leg->Draw();
C->Draw(); 